In [ ]:
pip install konlpy # 한국어 형태소 분석기 konlpy 설치

In [ ]:
pip install git+https://github.com/haven-jeon/PyKoSpacing.git # 딥러닝 띄어쓰기 패키지 설치

In [ ]:
pip install tomotopy # 토픽 모델 패키지 설치

In [ ]:
!sudo apt-get install -y fonts-nanum # 한글 폰트 설정
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

In [ ]:
# 라이브러리 불러오기

from konlpy.tag import *
from pykospacing import spacing
from nltk import Text
from matplotlib import pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from wordcloud import WordCloud
import numpy as np
import pandas as pd
import re, natsort, glob
import tomotopy as tp

km = Komoran() # 코모란 클래스 이용
plt.rc('font',family='NanumBarunGothic') # 한글 지정
model = tp.LDAModel(k=7,alpha=0.1,eta=0.01,min_df=1) # 토픽 모델 생성, 토픽 7개

In [ ]:
def han_fun(doc):
  nouns = km.nouns(doc)
  nouns = [noun for noun in nouns if len(noun)>=2] # 2글자 이상 필터링(남아있는 불용어 제거 목적)
  return nouns

text_list = natsort.natsorted(glob.glob('kp'+'*.txt'))

corpus,token_doc = [], []
for file in text_list:
  file_data = open(file,'r',encoding='cp949').readlines()[0]
  file_data = spacing(file_data) # 딥러닝을 이용한 띄어쓰기
  file_data = re.sub("\d+"," ",file_data) # 숫자 제거
  file_data = re.sub("연구|논문","",file_data) # 본 연구, 본 논문과 같은 불용어 제거
  file_data = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]','',file_data) # 특수문자 제거

  token = han_fun(file_data)
  model.add_doc(Text(token))
  corpus.append(file_data)

vector = TfidfVectorizer(tokenizer=han_fun) # TF-IDF Matrix 생성
vector1 = CountVectorizer(tokenizer=han_fun) # TF-IDF Matrix 생성

tdm = vector.fit_transform(corpus).toarray()
tdm1 = vector1.fit_transform(corpus).toarray()

column = vector.get_feature_names() # 컬럼명(단어) 얻음
column1 = vector1.get_feature_names() # 컬럼명(단어) 얻음
tdm = pd.DataFrame(tdm,index=text_list,columns=column) # Term-Document-Matrix 만들기
tdm1 = pd.DataFrame(tdm1,index=text_list,columns=column1) # Term-Document-Matrix 만들기

In [ ]:
model.train(200)
for i in range(model.k):
  res = model.get_topic_words(i, top_n=5)
  print('Topic #{}'.format(i), end='\t')
  print(', '.join(w for w, p in res))

In [ ]:
buckets = [[] for _ in range(model.k)]
for d in model.docs:
  buckets[d.get_topics(top_n=1)[0][0]].append(d)

In [ ]:
topic_model = pd.DataFrame(index=[0,1,2,3,4,5,6])

for i in range(len(model.docs)):
  a = pd.DataFrame(model.docs[i].get_topics(),columns=['index','doc_'+str(i+1)])
  a = a.set_index(a.columns[0])
  topic_model = topic_model.join(a)
topic_model

In [ ]:
count1= tdm1.sum(axis=0).sort_values(ascending=False)[0:20]
plt.figure(figsize=(15,7.5))
count1.plot(kind="barh",color='black')
plt.xlabel("빈도",fontsize=20,rotation=0) ;plt.ylabel("Words",fontsize=20,rotation=30)
plt.xticks(fontsize=15) ; plt.yticks(fontsize=15)
plt.title("Total Words(Count)",fontsize=30)

In [ ]:
count= tdm.sum(axis=0).sort_values(ascending=False)[0:20]
plt.figure(figsize=(15,7.5))
count.plot(kind="barh",color='black')
plt.xlabel("빈도",fontsize=20,rotation=0) ;plt.ylabel("Words",fontsize=20,rotation=30)
plt.xticks(fontsize=15) ; plt.yticks(fontsize=15)
plt.title("Total Words(TF-IDF)",fontsize=30)

In [ ]:
doc = tdm.loc[tdm.index[0]].sort_values(ascending=False) # 첫번째 문서만 분석 해보기
doc1 = tdm1.loc[tdm1.index[0]].sort_values(ascending=False) # 첫번째 문서만 분석 해보기

In [ ]:
plt.figure(figsize=(15,7.5))
doc1[:20].plot(kind='barh') # 빈도 그래프 그리기
plt.xlabel("빈도",fontsize=20,rotation=0) ;plt.ylabel("Words",fontsize=20,rotation=30,labelpad=50)
plt.xticks(fontsize=15) ; plt.yticks(fontsize=15)
plt.title("kp2004a(Count)",fontsize=30)

In [ ]:
plt.figure(figsize=(15,7.5))
doc[:20].plot(kind='barh') # 빈도 그래프 그리기
plt.xlabel("빈도",fontsize=20,rotation=0) ;plt.ylabel("Words",fontsize=20,rotation=30,labelpad=50)
plt.xticks(fontsize=15) ; plt.yticks(fontsize=15)
plt.title("kp2004a(TF-IDF)",fontsize=30)

In [ ]:
fontpath = '/usr/share/fonts/truetype/nanum/NanumGothic.ttf'
wc=WordCloud(font_path=fontpath,relative_scaling=0.2,background_color='white')
wc.generate_from_frequencies(dict(doc1[:30]))

plt.figure(figsize=(20,20))
plt.imshow(wc)
plt.axis('off')
plt.show()

In [ ]:
train_df = pd.read_csv("ratings_train.txt","\t") # 훈련 파일 불러오기
train_df = train_df.iloc[0:500] # 양이 너무 많기 때문에 500개로
test_df = pd.read_csv("ratings_test.txt","\t") # 테스트 파일 불러오기
test_df = test_df.iloc[0:10] # 역시 10개로rain_df = pd.read_csv("ratings_train.txt","\t") # 훈련 파일 불러오기
train_df = train_df.iloc[0:100] # 양이 너무 많기 때문에 100개로
test_df = pd.read_csv("ratings_test.txt","\t") # 테스트 파일 불러오기
test_df = test_df.iloc[0:50] # 역시 50개로

okt = Okt() # 토큰화를 시킬 라이브러리 로드

def tokenize(doc): # 토큰화 함수 정의
    return ['/'.join(t) for t in okt.pos(doc, norm=True, stem=True)]

okt = Okt() # 토큰화를 시킬 라이브러리 로드

def tokenize(doc): # 토큰화 함수 정의
    return ['/'.join(t) for t in okt.pos(doc, norm=True, stem=True)]

In [ ]:
train_df.isnull().any() # null값 제거
train_df['document'] = train_df['document'].fillna('')

test_df.isnull().any()
test_df['document'] = test_df['document'].fillna('')

In [ ]:
# 토큰화 과정 토큰화 과정
train_docs = [(tokenize(row[1]), row[2]) for row in train_df.values]
test_docs = [(tokenize(row[1]), row[2]) for row in test_df.values]
train_docs = [(tokenize(row[1]), row[2]) for row in train_df.values]
test_docs = [(tokenize(row[1]), row[2]) for row in test_df.values]

In [ ]:
FREQUENCY_COUNT = 100
tokens = [t for d in train_docs for t in d[0]]
text = Text(tokens, name='NMSC')
selected_words = [f[0] for f in text.vocab().most_common(FREQUENCY_COUNT)]
def term_frequency(doc):
    return [doc.count(word) for word in selected_words]
x_train = [term_frequency(d) for d,_ in train_docs]
x_test = [term_frequency(d) for d,_ in test_docs]

y_train = [c for _,c in train_docs]
y_test = [c for _,c in test_docs]

x_train = np.asarray(x_train).astype('float32')
x_test = np.asarray(x_test).astype('float32')

y_train = np.asarray(y_train).astype('float32')
y_test = np.asarray(y_test).astype('float32')

In [ ]:
import tensorflow as tf

model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(FREQUENCY_COUNT,)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [ ]:
#학습 프로세스 설정
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.001),
    loss=tf.keras.losses.binary_crossentropy,
    metrics=[tf.keras.metrics.binary_accuracy]
    )

In [ ]:
#학습 데이터로 학습
model.fit(x_train, y_train, epochs=20, batch_size=512)

In [ ]:
results = model.evaluate(x_test, y_test)

In [ ]:
review = "아주 재미 있어요"
token = tokenize(review)
tf = term_frequency(token)
data = np.expand_dims(np.asarray(tf).astype('float32'), axis=0)
float(model.predict(data))

In [ ]:
def predict_review(review):
    token = tokenize(review)
    tfq = term_frequency(token)
    data = np.expand_dims(np.asarray(tfq).astype('float32'), axis=0)
    score = float(model.predict(data))
    if(score > 0.5):
        print(f"{review} ==> 긍정 ({round(score*100)}%)")
    else:
        print(f"{review} ==> 부정 ({round((1-score)*100)}%)")

In [ ]:
predict_review("재미 정말 없어요")

In [ ]:
predict_review(review)

## 감정분석을 위해 Keras, Tensorflow와 같은 딥러닝 패키지가 필요했습니다
## 아래의 블로그를 참조하여 로직을 이해하는 쪽으로 공부하였습니다.

### https://devtimes.com/nlp-korea-movie-review